**This Is The prototype code for our Neural Network code**

***its coded in python using tensor flow in jupyter notebook to maintain data integrity***

Can be put into a python file, but needs lots of setup on the machine.

In [ ]:
import tensorflow as tf
import pandas as pd
import keras
from keras.utils import FeatureSpace

df = pd.read_json('Hango.Places.json')

#NHU SPECIFIC CODE HERE


################################################################
df.rename(columns={'_id': 'place_id'}, inplace=True)

df['place_id'] = df['place_id'].apply(lambda x: x['$oid'])


df.head()

,place_id,name,lat,lon,address,main_type,sub_types,rating,rating_amount,age,price,weblink
0,65c7c805d20df83fcf08aa6b,The Great White Hut,34.150018,-118.256291,"121 W California Ave, Glendale",Food,"[restaurant, food]",4.2,818,N,1,https://www.google.com/maps/place/?q=place_id:...
1,65c7c805d20df83fcf08aa6c,Chick-fil-A,34.049347,-118.259377,"660 S Figueroa St, Los Angeles",Food,"[restaurant, food]",4.2,1840,N,1,https://www.google.com/maps/place/?q=place_id:...
2,65c7c805d20df83fcf08aa6d,Melrose Food Co - by CloudKitchens,34.082243,-118.309399,"615 N Western Ave, Los Angeles",Food,"[restaurant, food]",4.5,133,N,2,https://www.google.com/maps/place/?q=place_id:...
3,65c7c805d20df83fcf08aa6e,Taco Bell,34.003267,-118.406905,"4416 Sepulveda Blvd, Culver City",Food,"[meal_takeaway, restaurant, food]",3.9,618,N,1,https://www.google.com/maps/place/?q=place_id:...
4,65c7c805d20df83fcf08aa6f,Grand Food Depot,34.014740,-118.278774,"358 W 38th St, Los Angeles",Food,"[restaurant, food]",4.6,258,N,2,https://www.google.com/maps/place/?q=place_id:...


In [164]:
import random

userdata ={
  "_id": {
    "$oid": "6567dcefba91df16f20f718d"
  },
  "username": "john_doe",
  "full_name": "John Doe",
  "email": "john@example.com",
  "age": 30,
  "address": {
    "street": "123 Main St",
    "city": "Anytown",
    "state": "CA",
    "zip_code": "12345",
    "country": "USA"
  }
}


def generateFakeReviews(id,age):
    
    reviewTable = pd.DataFrame()

    # Choose 10 random places for positive implicit feedback: 1
    positive_choices = random.sample(df['place_id'].tolist(), k=100)

    # Generate positive reviews and insert into reviewTable
    positive_reviews = [{'user_id': id, 'u_age': age, 'place': place, 'feedback': 1} for place in positive_choices]
    
    # Choose 5 random places for negative feedback: 0
    negative_choices = random.sample(df['place_id'].tolist(), k=75)

    # Generate negative reviews and insert into reviewTable
    negative_reviews = [{'user_id': id, 'u_age': age, 'place': place, 'feedback': 0} for place in negative_choices]
    
    reviewTable = reviewTable.append(positive_reviews, ignore_index=True)
    reviewTable = reviewTable.append(negative_reviews, ignore_index=True)

    return reviewTable

# Example usage
age = 25
uid = '6567dcefba91df16f20f718d'

generated_reviews = generateFakeReviews(id='6567dcefba91df16f20f718d', age=25)

generated_reviews.head(100)
    


C:\Users\memem\AppData\Local\Temp\ipykernel_17664\439151325.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reviewTable = reviewTable.append(positive_reviews, ignore_index=True)
C:\Users\memem\AppData\Local\Temp\ipykernel_17664\439151325.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reviewTable = reviewTable.append(negative_reviews, ignore_index=True)


,user_id,u_age,place,feedback
0,6567dcefba91df16f20f718d,25,65cbe9964d60dd8c13db52a7,1
1,6567dcefba91df16f20f718d,25,65cbc26bc72cccdf7a64543e,1
2,6567dcefba91df16f20f718d,25,65cbe853cbcec18078a36f19,1
3,6567dcefba91df16f20f718d,25,65cbe9914d60dd8c13db523c,1
4,6567dcefba91df16f20f718d,25,65cbe9154d60dd8c13db45bd,1
...,...,...,...,...
95,6567dcefba91df16f20f718d,25,65cbe825cbcec18078a36a57,1
96,6567dcefba91df16f20f718d,25,65cbe852cbcec18078a36efa,1
97,6567dcefba91df16f20f718d,25,65c7cd957f987a416ea4616f,1
98,6567dcefba91df16f20f718d,25,65cbe84fcbcec18078a36ead,1


In [165]:
#merge the two dataframes based on the place id to generate factor matrix
#drop features that are irrelevant to the model (distinction)
merged_db = pd.merge(generated_reviews, df, left_on='place', right_on='place_id', how='inner')
merged_db = merged_db.drop(columns=['user_id','place_id','place', 'name', 'address', 'weblink','price'])

#For now I think, we might be overfitting with every amount of data
merged_db['sub_types'] = merged_db['sub_types'].apply(lambda x: x[:3] if isinstance(x, list) else x)
merged_db[['subtype_1', 'subtype_2', 'subtype_3']] = pd.DataFrame(merged_db['sub_types'].tolist(), index=merged_db.index)

# Drop the original 'sub_types' column if needed
merged_db = merged_db.drop(columns=['sub_types'])
    
# Display the combined dataset
print(merged_db.head())
print(merged_db.size)


   u_age  feedback        lat         lon  main_type  rating  rating_amount  \
0     25         1  33.613384 -117.679997  Nightlife     4.2            430   
1     25         1  33.790114 -118.188881       Food     3.6           2434   
2     25         1  34.416927 -119.696150  Nightlife     3.5             83   
3     25         1  33.679853 -117.886696  Nightlife     4.2            777   
4     25         1  33.988638 -118.354489       Food     4.6           1167   

  age subtype_1      subtype_2    subtype_3  
0   Y       bar  meal_takeaway   restaurant  
1   N      cafe          store   restaurant  
2   Y      bars        chinese  restaurants  
3   Y       bar     night_club   restaurant  
4   N     store     restaurant         food  
1925


In [166]:
#turn into datasets

#divide the data in 2 sets train and validation
val_dataframe = df_combined.sample(frac=0.2, random_state=1337)
train_dataframe = df_combined.drop(val_dataframe.index)

#divide into features and the output value
val_ds_y = val_dataframe['feedback']
train_ds_y = train_dataframe['feedback']

val_ds_x = val_dataframe.drop(['feedback'], axis=1)
train_ds_x = train_dataframe.drop('feedback', axis=1)



#pardon if this is changed in the future this is only a month old not much documentation on it ;-;
feature_space = FeatureSpace(
    features={
        
        #"price": "integer_categorical", ##NEEDS TO BE PREPROCESSED SOMETIMES DICT SOMETIMES NOT
        
        "main_type": "string_categorical",
        "sub_type_1": "string_categorical",
        "sub_type_2": "string_categorical",
        "sub_type_3": "string_categorical",
        "age": "string_categorical",
        
        #"u_age": "float_discretized",
        #I'm not sure if we can even use age to predict.
        
        #normalized stuff
        "lat": "float_normalized",
        "lon": "float_normalized",
        "rating": "float_normalized",
        "rating_amount": "float_normalized",
    },
    #We make features that depend on both im not certain how reliable this will be.
    crosses=[("lat", "lon"), ("rating", "rating_amount")],
    crossing_dim=32,
    
    output_mode="concat",
)


#Here are our test and validation sets build the rest below